In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/10 21:43:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/10 21:44:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
customer_data = [
    {"customer_id": 1, "product_key": 5},
    {"customer_id": 2, "product_key": 6},
    {"customer_id": 3, "product_key": 5},
    {"customer_id": 3, "product_key": 6},
    {"customer_id": 1, "product_key": 6},
]

# Product data
product_data = [
    {"product_key": 5},
    {"product_key": 6},
]

# Create DataFrames
df_customer = spark.createDataFrame(customer_data)
df_product = spark.createDataFrame(product_data)

# Show DataFrames
df_customer.show()
df_product.show()

print(df_customer.dtypes)

+-----------+-----------+
|customer_id|product_key|
+-----------+-----------+
|          1|          5|
|          2|          6|
|          3|          5|
|          3|          6|
|          1|          6|
+-----------+-----------+

+-----------+
|product_key|
+-----------+
|          5|
|          6|
+-----------+

[('customer_id', 'bigint'), ('product_key', 'bigint')]


In [7]:
df_customer.createOrReplaceTempView("Customer")
df_product.createOrReplaceTempView("Product")

In [14]:
result = spark.sql(
    """          
select customer_id
from Customer where product_key in (select product_key from Product)
group by 1
having count(product_key) = (select count(*) from Product)

    """
)
result.show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+

